In [2]:
# Load Pandas library
import pandas as pd

#Load numpy library
import numpy as np

#import MatPlotLib
import matplotlib as mpl 

#Import PyPlot in MatPlotLib
import matplotlib.pyplot as plt 

#Import SciKitLearn decomposition PCA library
from sklearn.decomposition import PCA

#Import SciKitLearn dataset 
from sklearn import datasets as ds

#Helps create 80/20 split 
from sklearn.model_selection import ShuffleSplit

#Logistic Regression
from sklearn.linear_model import LogisticRegression

#Allows for meterics to be found after Logistic Regression
from sklearn import metrics as mt

#Create KFolds for CV_Object
from sklearn.model_selection import KFold

#Creates Standard Scaler Variables
from sklearn.preprocessing import StandardScaler

#Support Vector Machine
from sklearn.svm import SVC


#Seaborn 
import seaborn as sb

#Magics for inline code.
%matplotlib inline

#Create Zillow Data Frame using Read_CSV
# read in the csv file
dfZillowData = pd.read_csv('/users/cmirwin/desktop/Smu Data Sets/Data Mining/ZillowData/Properties_2016.csv') 

dfTrainingData = pd.read_csv('/users/cmirwin/desktop/Smu Data Sets/Data Mining/ZillowData/train_2016.csv')


#Create new data set that contains both the data zillow and the data to train against
dfFullTestData = pd.merge(dfTrainingData, dfZillowData, on='parcelid', how='left')

#Remove 2.9 million records from memory
del dfZillowData


#Get the headers to ensure all columns are included.
dfFullTestData.head(10)

/Users/cmirwin/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (22,32,34,49,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


parcelid  logerror transactiondate  airconditioningtypeid  \
0  11016594    0.0276      2016-01-01                    1.0   
1  14366692   -0.1684      2016-01-01                    NaN   
2  12098116   -0.0040      2016-01-01                    1.0   
3  12643413    0.0218      2016-01-02                    1.0   
4  14432541   -0.0050      2016-01-02                    NaN   
5  11509835   -0.2705      2016-01-02                    1.0   
6  12286022    0.0440      2016-01-02                    NaN   
7  17177301    0.1638      2016-01-02                    NaN   
8  14739064   -0.0030      2016-01-02                    NaN   
9  14677559    0.0843      2016-01-03                    NaN   

   architecturalstyletypeid  basementsqft  bathroomcnt  bedroomcnt  \
0                       NaN           NaN          2.0         3.0   
1                       NaN           NaN          3.5         4.0   
2                       NaN           NaN          3.0         2.0   
3                       NaN           NaN          2.0         2.0   
4                       NaN           NaN          2.5         4.0   
5                       NaN           NaN          4.0         4.0   
6                       NaN           NaN          1.0         2.0   
7                       NaN           NaN          2.5         3.0   
8                       NaN           NaN          1.0         2.0   
9                       NaN           NaN          2.0         2.0   

   buildingclasstypeid  buildingqualitytypeid         ...           \
0                  NaN                    4.0         ...            
1                  NaN                    NaN         ...            
2                  NaN                    4.0         ...            
3                  NaN                    4.0         ...            
4                  NaN                    NaN         ...            
5                  NaN                    1.0         ...            
6                  NaN                    7.0         ...            
7                  NaN                    NaN         ...            
8                  NaN                    NaN         ...            
9                  NaN                    NaN         ...            

   numberofstories  fireplaceflag  structuretaxvaluedollarcnt  \
0              NaN            NaN                    122754.0   
1              NaN            NaN                    346458.0   
2              NaN            NaN                     61994.0   
3              NaN            NaN                    171518.0   
4              2.0            NaN                    169574.0   
5              NaN            NaN                    880650.0   
6              NaN            NaN                     64549.0   
7              2.0            NaN                    107000.0   
8              NaN            NaN                     66834.0   
9              1.0            NaN                    109977.0   

   taxvaluedollarcnt  assessmentyear  landtaxvaluedollarcnt  taxamount  \
0           360170.0          2015.0               237416.0    6735.88   
1           585529.0          2015.0               239071.0   10153.02   
2           119906.0          2015.0                57912.0   11484.48   
3           244880.0          2015.0                73362.0    3048.74   
4           434551.0          2015.0               264977.0    5488.96   
5          2447951.0          2015.0              1567301.0   27126.57   
6           111521.0          2015.0                46972.0    2304.97   
7           306000.0          2015.0               199000.0    3745.50   
8           210064.0          2015.0               143230.0    2172.88   
9           190960.0          2015.0                80983.0    1940.26   

   taxdelinquencyflag  taxdelinquencyyear  censustractandblock  
0                 NaN                 NaN         6.037107e+13  
1                 NaN                 NaN                  NaN  
2                 NaN         

In [3]:
#We can see that a number of the data columns have NaN values that were assigned during import
#we choose to replace those values with the median values for the variable.

#Create new columns with highest missing values as True/False.
#print(dfMajorityOfDataMissing)

dfFullTestData['hasbasement'] = dfFullTestData[['basementsqft']].where(dfFullTestData[['basementsqft']].isnull(), 1).fillna(0).astype(int)

dfFullTestData['hasdeck'] = dfFullTestData[['yardbuildingsqft26']].where(dfFullTestData[['yardbuildingsqft26']].isnull(), 1).fillna(0).astype(int)

dfFullTestData['hasfireplace'] = dfFullTestData[['fireplaceflag']].where(dfFullTestData[['fireplaceflag']].isnull(), 1).fillna(0).astype(int)

dfFullTestData['hashottuborspa'] = dfFullTestData[['hashottuborspa', 'pooltypeid2']].where(dfFullTestData[['hashottuborspa','pooltypeid2']].isnull(), 1).fillna(0).astype(int)

dfFullTestData['haspatio'] = dfFullTestData[['yardbuildingsqft17']].where(dfFullTestData[['yardbuildingsqft17']].isnull(), 1).fillna(0).astype(int)

#Create Function for County Rename
def rename(row):
    if row['regionidcounty'] == 1286:
        val = 'Ventura County'
    elif row['regionidcounty'] == 3101:
        val = 'Los Angles County'
    else:
        val = 'Orange County'
    return val

dfFullTestData['countyname'] = dfFullTestData.apply(rename, axis=1)

print 'worked again'

worked again


In [4]:
#Create New data frame for everything
dfForModel = dfFullTestData.replace(to_replace='NaN',value= np.nan) 

#Get median values for based on original values
dfMedians = pd.DataFrame(dfFullTestData.median())

#Add Column Name to Medians DataFrame
dfMedians.columns = ['medianvalue']

#Create custom updates
dfColumns = pd.DataFrame(pd.Series([1.,0.,0.
                                    ,0.,0.,0.
                                    ,0.,0.,0.
                                    ,13.,5.,0.
                                    ,24.,0.,0.
                                    ,0.,0.,0.
                                    ,0.]
              , index=['numberofstories', 'bathroomcnt','bedroomcnt'
                      ,'fireplacecnt','fullbathcnt','garagecarcnt'
                      , 'poolcnt', 'roomcnt','unitcnt'
                      , 'heatingorsystemtypeid','airconditioningtypeid', 'garagetotalsqft'
                      , 'storytypeid', 'basementsqft', 'finishedsquarefeet13'
                      , 'finishedsquarefeet6','finishedsquarefeet15','finishedsquarefeet50'
                      , 'finishedfloor1squarefeet']))


#Set Column Name
dfColumns.columns = ['updatevalue']


#Create Merged Data set with values of both medians and custom
dfMerged = pd.concat([dfMedians, dfColumns], axis=1, join_axes=[dfMedians.index])

#Create new column of data that is custom values filled with median for any null
dfMerged['values'] = dfMerged['updatevalue'].fillna(dfMerged['medianvalue'])

#Apply median values to the overall data set
dfForModel = dfForModel.fillna(dfMerged['values'])

#Create smaller data set for descriptive stats section
dfDataForStats = dfForModel[['bathroomcnt',
                               'bedroomcnt',
                               'finishedfloor1squarefeet',
                               'calculatedfinishedsquarefeet',
                               'finishedsquarefeet15',
                               'lotsizesquarefeet',
                               'taxvaluedollarcnt',
                               'structuretaxvaluedollarcnt',
                               'landtaxvaluedollarcnt',
                               'taxamount']]

dfDataForStats.describe()


bathroomcnt    bedroomcnt  finishedfloor1squarefeet  \
count  90811.000000  90811.000000              90811.000000   
mean       2.266020      3.013974                101.768618   
std        1.016413      1.176174                398.691906   
min        0.000000      0.000000                  0.000000   
25%        2.000000      2.000000                  0.000000   
50%        2.000000      3.000000                  0.000000   
75%        3.000000      4.000000                  0.000000   
max       20.000000     16.000000               7625.000000   

       calculatedfinishedsquarefeet  finishedsquarefeet15  lotsizesquarefeet  \
count                  90811.000000          90811.000000       9.081100e+04   
mean                    1770.112310             93.409840       2.653193e+04   
std                      922.408353            508.302478       1.145533e+05   
min                        2.000000              0.000000       1.670000e+02   
25%                     1189.000000              0.000000       5.973000e+03   
50%                     1540.000000              0.000000       7.200000e+03   
75%                     2084.000000              0.000000       1.032850e+04   
max                    22741.000000          22741.000000       6.971010e+06   

       taxvaluedollarcnt  structuretaxvaluedollarcnt  landtaxvaluedollarcnt  \
count       9.081100e+04                9.081100e+04           9.081100e+04   
mean        4.569938e+05                1.796083e+05           2.778305e+05   
std         5.533113e+05                2.081280e+05           3.993632e+05   
min         2.200000e+01                1.000000e+02           2.200000e+01   
25%         1.999335e+05                8.178800e+04           8.291150e+04   
50%         3.428720e+05                1.320000e+05           1.929700e+05   
75%         5.383630e+05                2.093270e+05           3.440000e+05   
max         2.775000e+07                9.948100e+06           2.450000e+07   

           taxamount  
count   90811.000000  
mean     5975.374056  
std      6819.341095  
min        49.080000  
25%      2884.155000  
50%      4542.750000  
75%      6881.865000  
max    321936.090000

In [5]:
#Create dummy variables for all TypeId columns
#This section takes a long time due to the fact that we are creating 
#so many additional variables by 3 million records. 

#heatingorsystemtypeid
#Create dynamic Name
dummies = pd.get_dummies(dfForModel['heatingorsystemtypeid']).rename(columns=lambda x: 'HSTypeId_' + str(x))
#Join to Dataframe for Model
dfForModel = pd.concat([dfForModel, dummies], axis=1)

#Delete from model 
#dfForModel.drop(['heatingorsystemtypeid'], inplace=True, axis=1)

#Repeat of code above
#airconditioningtypeid
dummies = pd.get_dummies(dfForModel['airconditioningtypeid']).rename(columns=lambda x: 'ACTypeId_' + str(x))
dfForModel = pd.concat([dfForModel, dummies], axis=1)
#dfForModel.drop(['airconditioningtypeid'], inplace=True, axis=1)

#architecturalstyletypeid
dummies = pd.get_dummies(dfForModel['architecturalstyletypeid']).rename(columns=lambda x: 'ArchStyleTypeId_' + str(x))
dfForModel = pd.concat([dfForModel, dummies], axis=1)
#dfForModel.drop(['architecturalstyletypeid'], inplace=True, axis=1)

#decktypeid
dummies = pd.get_dummies(dfForModel['decktypeid']).rename(columns=lambda x: 'DeckTypeId_' + str(x))
dfForModel = pd.concat([dfForModel, dummies], axis=1)
#dfForModel.drop(['decktypeid'], inplace=True, axis=1)

#buildingclasstypeid
dummies = pd.get_dummies(dfForModel['buildingclasstypeid']).rename(columns=lambda x: 'BuildClassTypeId_' + str(x))
dfForModel = pd.concat([dfForModel, dummies], axis=1)
#dfForModel.drop(['buildingclasstypeid'], inplace=True, axis=1)

#typeconstructiontypeid
dummies = pd.get_dummies(dfForModel['typeconstructiontypeid']).rename(columns=lambda x: 'ConstructionTypeId_' + str(x))
dfForModel = pd.concat([dfForModel, dummies], axis=1)
#dfForModel.drop(['typeconstructiontypeid'], inplace=True, axis=1)

#buildingqualitytypeid
dummies = pd.get_dummies(dfForModel['buildingqualitytypeid']).rename(columns=lambda x: 'BuildingQualTypeId_' + str(x))
dfForModel = pd.concat([dfForModel, dummies], axis=1)
#dfForModel.drop(['buildingqualitytypeid'], inplace=True, axis=1)

#propertylandusetypeid
dummies = pd.get_dummies(dfForModel['propertylandusetypeid']).rename(columns=lambda x: 'LandUseTypeId_' + str(x))
dfForModel = pd.concat([dfForModel, dummies], axis=1)
#dfForModel.drop(['propertylandusetypeid'], inplace=True, axis=1)

#Create Function for County Rename
def garagecnt(row):
    if row['garagecarcnt'] > 0:
        val = 1
    else:
        val = 0
    return val

dfForModel['hasgarage'] = dfForModel.apply(garagecnt, axis=1)


#Find out what data types are in the Data set
dfDataTypes = pd.DataFrame(dfForModel)

#Create Numeric Only Dataframe
dfZillowDataNumeric = dfDataTypes.select_dtypes(['float64','int64'])
 
print 'complete again'

complete again


In [6]:
#Create Cross Validation object and Test and train values
# we want to predict if a house has a garage
if 'hasgarage' in dfZillowDataNumeric:
    
    #Get the values for each property
    actualValues = dfZillowDataNumeric['hasgarage'].values 
    
    # get rid of the hasgarage variable
    del dfZillowDataNumeric['hasgarage'] 

    #delete any variables that give the model the possible anwser
    del dfZillowDataNumeric['garagecarcnt']

    # Gather all values to use for prediction
    additionalVars = dfZillowDataNumeric.values    

print 'groups created'

# Create cross validation with KFolds object
num_cv_iterations = 3
#Create Cross Validation objects using folds
cv_object = KFold(n_splits=num_cv_iterations, random_state=None, shuffle=True)
cv_object.get_n_splits(additionalVars)

for train_index, test_index in cv_object.split(additionalVars):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = additionalVars[train_index], additionalVars[test_index]
    y_train, y_test = actualValues[train_index], actualValues[test_index]
 
print 'Test and Train Data Created'
#This section scales the data in order to create a more useable data set
# scale attributes by the training set
stdsclrObj = StandardScaler()

# find scalings for each column that make this zero mean and unit std
#We use the training data to help fit the standard dev
stdsclrObj.fit(X_train) # find scalings for each column that make this zero mean and unit std

# to transform new feature data
# apply to training
X_train_scaled = stdsclrObj.transform(X_train)

# apply those means and std to the test set (without snooping at the test set values)
X_test_scaled = stdsclrObj.transform(X_test) 

# Create a reusable logisitic regression object
# the variable C is for Cost, penalty is actually a lowercase L and has 
# other options l1 and l2
logRegression = LogisticRegression(penalty='l2', C=1.0, class_weight=None) # get object

# Using the CV Object we iterate through the folds.
for iter_num, (train_indices, test_indices) in enumerate(cv_object.split(X_train_scaled,y_train)):
    logRegression.fit(X_train_scaled[train_indices],y_train[train_indices])  # train object
    y_hat = logRegression.predict(X_train_scaled[test_indices]) # get test set precitions

    # print the accuracy and confusion matrix 
    print("====Iteration",iter_num + 1," ====")
    print("accuracy", mt.accuracy_score(actualValues[test_indices],y_hat)) 
    print("confusion matrix\n",mt.confusion_matrix(actualValues[test_indices],y_hat))

groups created
('TRAIN:', array([    0,     1,     2, ..., 90807, 90808, 90810]), 'TEST:', array([    8,    11,    13, ..., 90804, 90806, 90809]))
('TRAIN:', array([    0,     3,     5, ..., 90806, 90808, 90809]), 'TEST:', array([    1,     2,     4, ..., 90805, 90807, 90810]))
('TRAIN:', array([    1,     2,     4, ..., 90807, 90809, 90810]), 'TEST:', array([    0,     3,     5, ..., 90793, 90802, 90808]))
Test and Train Data Created
('====Iteration', 1, ' ====')
('accuracy', 0.56127050195728656)
('confusion matrix\n', array([[9044, 4454],
       [4400, 2283]]))
('====Iteration', 2, ' ====')
('accuracy', 0.55678889990089198)
('confusion matrix\n', array([[9061, 4454],
       [4490, 2175]]))
('====Iteration', 3, ' ====')
('accuracy', 0.55822596630327059)
('confusion matrix\n', array([[9024, 4581],
       [4334, 2241]]))


In [7]:
#Print results of last run.
# combine attributes
coefValues = zip(logRegression.coef_.T,dfZillowDataNumeric.columns) 

# sort these attributes and spit them out
coefValues = pd.DataFrame(sorted(coefValues))

#zip_vars.columns = ['values', 'names']

#keep all columns from dataframe with absolute values of .25 
dfValuesGTfive = coefValues.loc[abs(coefValues[0]) > .25]

#Reset Index from removing data
dfValuesGTfive = dfValuesGTfive.reset_index(drop=True)

#Remove any duplicate values from multiple runs
dfValuesGTfive = dfValuesGTfive.drop_duplicates([1], keep='last')

# iterate over the Variable values
#Get weight values
weights = dfValuesGTfive[0] 
#Get Variable Names
variable_names = dfValuesGTfive[1]
#Get Index count
indexes = dfValuesGTfive.index
#Combined the data set and then print out results
for index, weight, name in zip(indexes, weights,variable_names):
    print('index value of ', index , 'Variable Name: ', name, 'has weight of', weight[0])
    
#Due to the large number of variables we choose an absolute value of .15 as a cut off
#Show all variables with an absolute value of greater than .15
plt.style.use('ggplot')
#The commented line below give the correct labels, but not the correct values.
#plotweights = pd.Series(dfValuesGTfive[0].astype(float), index=variable_names)
plotweights = pd.Series(dfValuesGTfive[0].astype(float))
plotweights.plot(kind='bar')
plt.show()

('index value of ', 0, 'Variable Name: ', 'regionidcounty', 'has weight of', -2.9348072494341091)
('index value of ', 1, 'Variable Name: ', 'structuretaxvaluedollarcnt', 'has weight of', -1.9398825053434809)
('index value of ', 2, 'Variable Name: ', 'calculatedbathnbr', 'has weight of', -1.2899254693237985)
('index value of ', 3, 'Variable Name: ', 'finishedsquarefeet15', 'has weight of', -1.0936923723403493)
('index value of ', 4, 'Variable Name: ', 'LandUseTypeId_261.0', 'has weight of', -1.0404202645897802)
('index value of ', 5, 'Variable Name: ', 'lotsizesquarefeet', 'has weight of', -0.98408731661970827)
('index value of ', 6, 'Variable Name: ', 'finishedsquarefeet50', 'has weight of', -0.70011917584053762)
('index value of ', 7, 'Variable Name: ', 'fullbathcnt', 'has weight of', -0.62637368045124053)
('index value of ', 8, 'Variable Name: ', 'roomcnt', 'has weight of', -0.57479344423701328)
('index value of ', 9, 'Variable Name: ', 'finishedsquarefeet12', 'has weight of', -0.566